#### Задачи:
=============

* Извлечь utm-метки из колонки referrer, в итоговом датасете добавить новые колонки: utm_source, utm_medium, utm_campaign, utm_term, utm_content с соответствующими значениями.

* Внутри utm_content у некоторых ссылок есть параметр crid, за которым после символа | следует цифра (например: crid|17) – это номер рекламного баннера. Необходимо добавить колонку creative_id в датасет с этими номерами в сессиях в которых они есть.

* В колонке visitor_device необходимо избавится от словарей, оставив только значение поля “value”.

* Необходимо добавить колонку channel значение которой будет зависеть от utm_source и referrer:
    * Если referrer пуст, тогда channel = “direct”
    * Если referrer = “https://www.google.com/” тогда channel = “organic”
    * Если utm_source не пуст то берем значение оттуда
    * Добавить колонку week_day в которую на основе даты и visit_date необходимо поместить день недели в формате: Monday, Tuesday..



In [1]:
import pandas as pd
import numpy as np

In [166]:
df = pd.read_csv (r'/Users/soficodanilyantz/Downloads/dataset_visitors.csv')
#dataset_visitors представляет из себя сэмпл с сессиями сайта, где
#visit_datetime  – дата и время визита
#referrer – ссылка перехода на сайт
#visitor_device – устройство посетителя
#is_lead – если True то посетитель оставил заявку во время ссесии

df[df['referrer'].str.len()> 0]

,visit_date,referrer,visitor_device,is_lead
4,2021-03-17 13:25:55.861624,https://www.google.com/,"{""value"": ""PC"", ""value_id"": ""desktop""}",True
7,2021-03-10 07:16:53.577870,https://callgear.ae/vpbx/?utm_source=google&ut...,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN
8,2021-03-23 20:03:21.098973,https://www.google.com/,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN
10,2021-03-11 15:27:48.953132,https://www.google.com/,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",NaN
11,2021-03-17 02:19:57.740003,https://www.google.com/,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",True
...,...,...,...,...
1251,2021-03-17 21:33:09.767438,https://www.google.com/,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN
1252,2021-03-16 12:44:25.596576,https://callgear.ae/control-employees-lp-oae/m...,"{""value"": ""Mobile"", ""value_id"": ""mobile""}",True
1255,2021-03-30 14:36:37.681928,https://www.google.com/,"{""value"": ""PC"", ""value_id"": ""desktop""}",True
1256,2021-03-06 12:33:40.368315,https://www.google.com/,"{""value"": ""PC"", ""value_id"": ""desktop""}",NaN


In [94]:
#для парсинга UTM меток из колонки referrer будем использовать построчный обход по всей длине датафрейма. 

utm = ['utm_source','utm_medium','utm_campaign','utm_term','utm_content']
for j in utm:
    utm_list = []
    f_val = j+'='
    for i in range(len(df['referrer'])):
        val = str(df.loc[i,'referrer'])
        if len(val) >len('https://www.google.com/'): #поиск строк с адресной строкой
            utm_start = val.find(f_val) #точка начала метки
            if utm_start < 1:
                utm_list.append(np.nan)
                continue
            utm_stop = val.find('&', utm_start) #точки конца метки
            utm_list.append(val[utm_start+len(f_val):utm_stop]) #слайсинг поля 'refferer' и добавление в список
        else:
            utm_list.append(np.nan)
    t[j]=utm_list #т.к длина списка соотвествует длине датафрейма, то мы можем добавить его в качестве новой колонки
#t[t['utm_source'].notnull()]

In [95]:
#создаем колонку creative_id 
utm_list = []
f_val = 'crid|'
for i in range(len(df['referrer'])):
    val = str(t.loc[i,'utm_content'])
    if len(val) >1: #поиск ненулевых строк 
        utm_start = val.find(f_val)
        if utm_start < 1: 
            utm_list.append(np.nan)
            continue
        utm_stop = val.find('|', utm_start+len(f_val))
        utm_list.append(val[utm_start+len(f_val):utm_stop]) #слайсинг
    else:
        utm_list.append(np.nan)
t['creative_id']=utm_list 
#t[t['creative_id'].notnull()]

In [102]:
t['visitor_device'] = df['visitor_device']
#создаем колонку visitor_device, воспользуемся предыдущим алгоритмом для utm меток
utm_list = []
f_val = '"value": "'
for i in range(len(df['referrer'])):
    val = str(t.loc[i,'visitor_device'])
    if len(val) >1: #поиск ненулевых строк 
        utm_start = val.find(f_val)
        if utm_start < 1: 
            utm_list.append(np.nan)
            continue
        utm_stop = val.find('"', utm_start+len(f_val))
        utm_list.append(val[utm_start+len(f_val):utm_stop]) 
    else:
        utm_list.append(np.nan)
t['visitor_device_new']=utm_list


In [103]:
t = t.drop(columns='visitor_device')
t = t.rename(columns={"visitor_device_new": "visitor_device"})


,visit_date,referrer,is_lead,utm_source,utm_medium,utm_campaign,utm_term,utm_content,creative_id,visitor_device
0,2021-03-27 10:38:19.482951,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,PC
1,2021-03-10 12:50:36.680002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mobile
2,2021-03-05 19:53:09.428238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PC
3,2021-03-08 10:15:12.315905,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,PC
4,2021-03-17 13:25:55.861624,https://www.google.com/,True,NaN,NaN,NaN,NaN,NaN,NaN,PC
...,...,...,...,...,...,...,...,...,...,...
1253,2021-03-16 12:39:11.964890,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,Mobile
1254,2021-03-18 07:55:19.214361,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,PC
1255,2021-03-30 14:36:37.681928,https://www.google.com/,True,NaN,NaN,NaN,NaN,NaN,NaN,PC
1256,2021-03-06 12:33:40.368315,https://www.google.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PC


In [127]:
t['channel'] = np.where(t['referrer'].isnull(),\
                        'direct',\
                        np.where(t['referrer'] == 'https://www.google.com/','organic',t['utm_source']))

#np.where - работает по принципу (кондиция,значение если True,значение если False)
            

In [170]:
#Добавление дня недели
t['day_of_week'] = pd.to_datetime(t['visit_date'].str.slice(0,10))
t['day_of_week'] = t['day_of_week'].dt.day_name()
t[t['referrer'].str.len()> 40]

,visit_date,referrer,is_lead,utm_source,utm_medium,utm_campaign,utm_term,utm_content,creative_id,visitor_device,channel,day_of_week
7,2021-03-10 07:16:53.577870,https://callgear.ae/vpbx/?utm_source=google&ut...,NaN,google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,voip cloud,tid|kwd-13704117858|cid|12187113246|aid|494770...,NaN,PC,google,Wednesday
17,2021-03-12 16:38:31.863260,https://callgear.ae/control-employees-lp-oae/m...,True,facebook,cpm,FB_UAE_ALL_CONVERSION_GN_AG_J35|23848562300580067,NaN,cid|23848562300580067|gid|23848562301270067|ai...,91,Mobile,facebook,Friday
39,2021-03-26 12:50:36.529962,https://callgear.ae/vpbx/?utm_source=google&ut...,NaN,google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,voip cloud,tid|kwd-13704117858|cid|12187113246|aid|494770...,NaN,PC,google,Friday
41,2021-03-18 08:23:59.871134,https://callgear.ae/pricing/?cm_id=12187113246...,NaN,google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,system pabx,tid|kwd-351055527567|cid|12187113246|aid|50919...,NaN,PC,google,Thursday
44,2021-03-13 12:11:57.159766,https://callgear.ae/vpbx/?utm_source=google&ut...,True,google,cpc,GAW_UAE_DU_SEARCH_VP_TR_Key_NewPage|12187113246,phone virtual,tid|kwd-299273384361|cid|12187113246|aid|50919...,NaN,PC,google,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...
1245,2021-03-02 21:52:28.523109,https://callgear.ae/control-employees-lp-oae/m...,True,facebook,cpm,FB_UAE_ALL_CONVERSION_GN_AG_J35|23848562300580067,NaN,cid|23848562300580067|gid|23848562301220067|ai...,98,Mobile,facebook,Tuesday
1246,2021-03-25 23:08:05.975038,https://callgear.ae/landline-uae-number/up/?ut...,True,facebook,cpm,FB_UAE_ALL_CONVERSION_GN_AG_J34|23848680267780067,NaN,cid|23848680267780067|gid|23848680267810067|ai...,108,Mobile,facebook,Thursday
1248,2021-03-24 01:04:22.813035,https://callgear.ae/missed-calls-lp-oae-2/?utm...,True,facebook,cpm,FB_UAE_DU_CONVERSION_GN_AG_J35|23848827103510067,NaN,cid|23848827103510067|gid|23848830511000067|ai...,111,Mobile,facebook,Wednesday
1250,2021-03-18 18:34:20.057726,https://callgear.ae/missed-calls-lp-oae-2/?utm...,True,facebook,cpm,FB_UAE_DU_CONVERSION_GN_AG_J35|23848827103510067,NaN,cid|23848827103510067|gid|23848830511000067|ai...,111,Mobile,facebook,Thursday


In [153]:
#импортируем итоговый датафрейм в csv
from pathlib import Path  
filepath = Path('/Users/soficodanilyantz/Downloads/dataset_visitors_Rastoropov.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
t.to_csv(filepath) 